# Hugging Face GPT-Neo 2.7B Model Inference

This notebook demonstrates how to deploy Hugging Face GPT-Neo 2.7B transformer models using Hugging Face Text Generation Inference (TGI) Deep Learning Container on Amazon SageMaker.

GPT-Neo 2.7B is a transformer model designed using EleutherAI's replication of the GPT-3 architecture. GPT-Neo refers to the class of models, while 2.7B represents the number of parameters of this particular pre-trained model.

## Setup

### Install the SageMaker Python SDK

First, make sure that the latest version of SageMaker SDK is installed.

In [ ]:
%pip install "sagemaker>=2.163.0"

## Setup account and role

Then, we import the SageMaker python SDK and instantiate a sagemaker_session which we use to determine the current region and execution role.

In [ ]:
import sagemaker
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
import time

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()

## Retrieve the LLM Image URI

We use the helper function get_huggingface_llm_image_uri() to generate the appropriate image URI for the Hugging Face Large Language Model (LLM) inference.

The function takes a required parameter backend and several optional parameters. The backend specifies the type of backend to use for the model, the values can be "lmi" and "huggingface". The "lmi" stands for SageMaker LMI inference backend, and "huggingface" refers to using Hugging Face TGI inference backend.

In [ ]:
image_uri = get_huggingface_llm_image_uri(backend="huggingface", region=region)  # or lmi

## Create the Hugging Face Model

Next we configure the model object by specifying a unique name, the image_uri for the managed TGI container, and the execution role for the endpoint. Additionally, we specify a number of environment variables including the HF_MODEL_ID which corresponds to the model from the HuggingFace Hub that will be deployed, and the HF_TASK which configures the inference task to be performed by the model.

You should also define SM_NUM_GPUS, which specifies the tensor parallelism degree of the model. Tensor parallelism can be used to split the model across multiple GPUs, which is necessary when working with LLMs that are too big for a single GPU. Here, you should set SM_NUM_GPUS to the number of available GPUs on your selected instance type. For example, in this tutorial, we set SM_NUM_GPUS to 4 because our selected instance type ml.g4dn.12xlarge has 4 available GPUs.

Additionally, we could reduce the memory footprint of the model by setting the HF_MODEL_QUANTIZE environment variable to true.

In [ ]:
model_name = "gpt-neo-27B-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

hub = {"HF_MODEL_ID": "EleutherAI/gpt-neo-2.7B", "HF_TASK": "text-generation", "SM_NUM_GPUS": "4"}

model = HuggingFaceModel(name=model_name, env=hub, role=role, image_uri=image_uri)

## Creating a SageMaker Endpoint

Next we deploy the model by invoking the deploy() function. Here we use an ml.g4dn.12xlarge instance which come with 4 NVIDIA A10 GPUs. By setting the SM_NUM_GPUS environment variable to 4 in the last code block, we indicate that this model should be sharded across all 4 GPU devices.

In [ ]:
predictor = model.deploy(
    initial_instance_count=1, instance_type="ml.g5.12xlarge", endpoint_name=model_name
)

## Running Inference

Once the endpoint is up and running, we can evaluate the model using the predict() function.

In [ ]:
input_data = {
    "inputs": "The diamondback terrapin was the first reptile to",
    "parameters": {"do_sample": True, "max_new_tokens": 100, "temperature": 0.7, "watermark": True},
}

predictor.predict(input_data)

## Cleaning Up
After you've finished using the endpoint, it's important to delete it to avoid incurring unnecessary costs.

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()

In this tutorial, we used a TGI container to deploy EleutherAI/gpt-neo-2.7B model using 4 GPUs on a SageMaker ml.g5.12xlarge instance. With Hugging Face's Text Generation Inference and SageMaker Hosting, you can easily host large language models like GPT-NeoX, flan-t5-xxl, and LLaMa.